In [2]:
import graphlab
# Downloads the data from S3 if you haven't already.
import os
if os.path.exists('stack_overflow'):
    sf = graphlab.SFrame('stack_overflow')
else:
    sf = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/stack_overflow')
    sf.save('stack_overflow')

[INFO] This commercial license of GraphLab Create is assigned to yuanfeng.wang@livingsocial.com.

[INFO] Start server at: ipc:///tmp/graphlab_server-51487 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1440966049.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/dir_archive.ini to /var/tmp/graphlab-yuanfengwang/51487/000000.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/objects.bin to /var/tmp/graphlab-yuanfengwang/51487/000001.bin
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/m_74b0dc51.frame_idx to /var/tmp/graphlab-yuanfengwang/51487/000002.frame_idx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/m_74b0dc51.sidx to /var/tmp/graphlab-yuanfengwang/51487/000003.sidx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/m_74b0dc51.0000 to /var/tmp/graphlab-yuanfengwang/51487/000004.0000
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/m_74b0dc51.0001 to /var/tmp/graphlab-yuanfengwang/51487/000005.0001
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stack_overflow/m_74b0dc51.0002 to /var/tmp/graphlab-yuanfengwang/51487/000006.

In [18]:
sf.num_rows()

3643508

In [7]:
#check missing values
for i in sf.column_names():
    sf[i] = sf[i].apply(lambda x: None if x == '' else x)

In [19]:
sf.column_names()

['PostId',
 'PostCreationDate',
 'OwnerUserId',
 'OwnerCreationDate',
 'ReputationAtPostCreation',
 'OwnerUndeletedAnswerCountAtPostTime',
 'Title',
 'BodyMarkdown',
 'PostClosedDate',
 'OpenStatus',
 'tags']

In [12]:
sf.groupby('OpenStatus',operations={'count': graphlab.aggregate.COUNT()})

OpenStatus,count
",jquery,,,,,,open",1
off topic,20825
",silverlight,ivalueconverter,,,,,open ...",1
open,3554596
",python,,,,,,open",1
",escaping,key,event-handling,textbox,,,open ...",1
not constructive,20878
",c#,.net,javascript,asp.net,datagrid,,open ...",1
",jquery,json,associative-array,,,,open ...",1
",c#,regex,,,,,open",1


In [10]:
# Come up with criteria for a "double post" and filter out duplicates.
sf.groupby?

In [14]:
# This is just one of many definitions of a "double post".  If I was to predict
# whether a question will be closed, I would not filter these out.
tmp = sf.groupby(['Title','OwnerUserId', 'BodyMarkdown'], {'PostId':graphlab.aggregate.SELECT_ONE('PostId')})
filtered = sf.filter_by(tmp['PostId'], 'PostId')
print len(sf)-len(filtered)

142


In [17]:
sf = sf.pack_columns(['Tag1','Tag2','Tag3','Tag4','Tag5'], new_column_name='tags')

ValueError: Current SFrame has no column called 'Tag1'.

In [21]:
#Display the top 10 question askers.
sf.groupby(['OwnerUserId'], {'post_count':graphlab.aggregate.COUNT()}).topk('post_count', k=10)

OwnerUserId,post_count
39677,1781
4653,1437
34537,1423
179736,1234
149080,1190
117700,1172
84201,1164
434051,1093
325418,995
4639,974


In [22]:
#Find the highest rated asker for the 'not constructive' question type.
sf['ReputationAtPostCreation'] = sf['ReputationAtPostCreation'].astype(int)
notc = sf[sf['OpenStatus'] == 'not constructive']
notc.topk('ReputationAtPostCreation', k=1)

Columns:
	PostId	str
	PostCreationDate	str
	OwnerUserId	str
	OwnerCreationDate	str
	ReputationAtPostCreation	int
	OwnerUndeletedAnswerCountAtPostTime	str
	Title	str
	BodyMarkdown	str
	PostClosedDate	str
	OpenStatus	str
	tags	list

Rows: 1

Data:
+----------+---------------------+-------------+---------------------+--------------------------+
|  PostId  |   PostCreationDate  | OwnerUserId |  OwnerCreationDate  | ReputationAtPostCreation |
+----------+---------------------+-------------+---------------------+--------------------------+
| 11058483 | 06/15/2012 21:32:26 |    16417    | 09/17/2008 17:02:31 |          98265           |
+----------+---------------------+-------------+---------------------+--------------------------+
+-------------------------------+--------------------------------+
| OwnerUndeletedAnswerCountA... |             Title              |
+-------------------------------+--------------------------------+
|              1000             | PHP - Justify string algorithm |
+-------------------------------+--------------------------------+
+-------------------------------+---------------------+------------------+
|          BodyMarkdown         |    PostClosedDate   |    OpenStatus    |
+-------------------------------+---------------------+------------------+
| Just tanked a job intervie... | 06/22/2012 13:01:37 | not constructive |
+-------------------------------+---------------------+------------------+
+-------------------------------+
|              tags             |
+-------------------------------+
| [php, algorithm, None, Non... |
+-------------------------------+
[1 rows x 11 columns]